# Inteligencia Artificial
# Clase 8 - Redes Neuronales con Keras

## Clasificando críticas de películas: un ejemplo de clasificación binaria

Los problemas de clasificación de dos clases, o clasificación binaria, tal vez sean el tipo de problema de *machine learning* más ampliamente aplicado.

El objetivo de esta práctica es construir una red neuronal para clasificar críticas de películas en críticas "positivas" y críticas "negativas", sólo basados en el contenido del texto de las críticas.

## The IMDB dataset

Vamos a trabajar con el "IMDB dataset", un set de 50.000 críticas altamente polarizadas de la Internet Movie Data Base (IMDB). Se separan en un set de 25.000 críticas para training y 25.000 críticas para testing, cada una consistente en 50% de casos positivos y 50% de casos negativos.

El dataset IMDB viene con Keras. Ya ha sido preprocesado: las críticas (secuencia de palabras) han sido convertidas en secuencias de valores numéricos, donde cada entero representa una palabra específica en un diccionario.

El código que sigue va a cargar el dataset (cuando lo corras por primera vez se van a descargar unos 80 mb en tu máquina):

In [0]:
import numpy as np
from keras.datasets import imdb

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

El argumento `num_words=10000` significa que sólo se van a mantener las 10.000 palabras más frecuentemente empleadas en el set de training. Palabras raras van a ser descartadas. Esto nos permite trabajar con un vector de información de tamaño manejable.

Las variables `train_data` y `test_data` son listas de críticas, cada crítica es a su vez una lista de índices de palabras (que codifican una secuencia de palabras).
`train_labels` y `test_labels` son listas de 0s y 1s, donde 0 es "negativo" y 1 es "positivo".

In [0]:
def get_id_to_word(INDEX_FROM=3):
  """
  Retorna un diccionario que vamos a usar para convertir vectores de samples
  en palabras.
  """
  word_to_id = imdb.get_word_index()
  word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
  word_to_id["<PAD>"] = 0
  word_to_id["<START>"] = 1
  word_to_id["<UNK>"] = 2
  id_to_word = {value:key for key,value in word_to_id.items()}
  return id_to_word, word_to_id


def vector_to_words(vect, id_to_word):
  return ' '.join(id_to_word[id] for id in vect)


id_to_word, word_to_id = get_id_to_word()


In [0]:
# Completar aquí con el codigo para mostrar el primer elemento de
# train_data como vector de enteros.
## TU CÓDIGO AQUÍ ##

In [0]:
# Usar vector_to_words para convertir ese vector en texto legible.
## TU CÓDIGO AQUÍ ##

In [0]:
# Imprimir el primer elemento de train_labels.
## TU CÓDIGO AQUÍ ##

Como nos restringimos a mantener las 10.000 palabras más frecuentes, ningún índice de palabra va a exceder 10.000:

In [0]:
#Chequeamos que el tamaño del vector de palabras no exceda los 10000 elementos.
## TU CÓDIGO AQUÍ ##


## Preparando la información

No se puede alimentar a una red neuronal con listas de enteros. Necesitamos convertir nuestras listas en tensores. Hay dos maneras para hacer esto:

* Podemos rellenar nuestras listas para que todas tengan el mismo largo, y convertirlas en un tensor de enteros de forma `(samples, word_indices)`, 
luego usar como primera capa de nuestra red una capa capaz de manejar esos tensores de enteros (en inglés se la conoce como `Embedding` layer).

* Podemos hacer *one-hot-encoding* de nuestras listas para convertirlas en vectores de 0s y 1s. Concretamente, esto significaría, por ejemplo, convertir una secuencia `[3, 5]` en un vector 10.000-dimensional que sería todo 0s excepto por los índices 3 y 5, que serían 1s. Luego podríamos usar como primera capa una capa densa, capaz de manejar vectores de datos de punto flotante.

Vamos a ir por la segunda solución. Vectoricemos nuestros datos, que vamos a hacerlo manualmente para tener máxima claridad:

In [0]:
def vectorize_sequences(sequences, dimension=10000):
    # Creo una matriz de ceros de dimensión (len(sequences), dimension)
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.  # seteo índices específicos a 1
    return results

# Nuestros datos vectorizados de training
x_train = vectorize_sequences(train_data)

# Nuestros datos vectorizados de test
x_test = vectorize_sequences(test_data)

Así se ven nuestros ejemplos ahora:

In [0]:
# Vemos como queda el primer elemento de x_train
x_train[0]

También tenemos que vectorizar las etiquetas:

In [0]:
# Nuestras etiquetas vectorizadas
y_train = np.asarray(train_labels).astype('float32')
y_test = np.asarray(test_labels).astype('float32')

Ahora nuestra información está lista para alimentar la red neuronal.

## Construyendo la red

Nuestros datos de entrada son simplemente vectores, y nuestras etiquetas escalares (1s y 0s): esta es la forma más sencilla de setup que se puede encontrar. 
Una red que performe bien en un problema así sería simplemente un conjunto de capaz completamente-conectadas (`Dense`) con activación `relu` : `Dense(16, 
activation='relu')`

El argumento que le pasamos a cada capa densa (16) es el número de "unidades ocultas" de la capa. ¿Qué es una "unidad oculta"? Es una dimensión en el espacio de representación de la capa.
Una capa `Densa` con una activación `relu` implementa la siguiente cadena de operación de tensor:

`output = relu(dot(W, input) + b)`

Tener 16 unidades ocultas significa que la matriz de pesos `W` va a tener forma `(input_dimension, 16)`, i.e. el producto escalar con `W` va a proyectar la información de entrada en un espacio de representación 16-dimensional (y luego va a agregar el vector de sesgo `b` y aplicar la operación `relu`). Intuitivamente, podés entender la dimensionalidad del espacio de representación como "cuanta libertad le estás dejando a la red tener al aprender representaciones internas". 
Tener más unidades ocultas (un espacio de representación de mayor dimensionalidad) permite a la red aprender representaciones más complejas, pero convierte a la red en computacionalmente más cara y puede llevar a aprender patrones no deseados (patrones que pueden permitir mejorar la *performance* en la información de train pero no de test).

Hay dos decisiones de arquitectura que tienen que ser hechas en una pila de capaz densas:

* Cuántas capas usar.
* Cuántas "unidades ocultas" elegir para cada capa.

Por el momento, vamos a elegir arbitrariamente 2 capaz intermedias con 16 unidades ocultas cada una, y una tercera capa que va a productir la predicción que respecta al sentimiento de la crítica.

Las capas intermedias van a usar `relu` como sus funciones de activación, y la capa final va a usar una función de activación sigmoidal para producir una probabilidad (score entre 0 y 1, que indica qué tan probable la muestra es de tener clase "1", i.e, qué tan probable es que la crítica sea positiva).
Una `relu` (por sus siglas en inglés, "rectified linear unit") es una función destina a devolver cero valores negativos. mientras que la sigmoidal aprieta los valores en el intervalo `[0, 1]`, dando como producto algo que puede ser interpretado como probabilidad.

Así se ve nuestra red:

![3-layer network](https://s3.amazonaws.com/book.keras.io/img/ch3/3_layer_network.png)

Y la implementación de Keras:

In [0]:
# Creamos un modelo simple
# Se sugiere usar dos capas de 16 neuronas con activacion `relu`.
# Ver que la capa de salida tenga una neurona y funcion de activacion 'sigmoid'.

from keras import models, layers

## TU CÓDIGO AQUÍ ##


Por último, necesitamos elegir una función de pérdida y un optimizador. Dado que estamos enfrentando un problema de clasificación binaria y que el producto de nuestra red es una probabilidad (terminamos nuestra red con una capa de una sola unidad con una función de activación), es que lo mejor es elegir la pérdida `binary_crossentropy`. 
No es la única opción viable: también podrías usar, por ejemplo, `mean_squared_error`. Pero la entropía cruzada es usualmente la mejor opción cuando estás enfrentando modelos que dan como *output* probabilidades. La entropía cruzada es una medida del campo de la teoría de la Información, que mide la "distancia" entre distribución de probabilidad, o en nuestro caso, entre la distribución real y nuestras predicciones.

Éste es el paso donde configuramos nuestro modelo con el optimizador `rmsprop` y la función de pérdida `binary_crossentropy`. Nótese que vamos a monitorear el accuracy en training.

In [0]:
# Compilar el modelo usando el optimizador rmsprop, con la pérdida
# correspondiente y grabando la métrica de accuracy.

## TU CÓDIGO AQUÍ ##

Pasamos nuestro optimizador, función de pérdida y métricas como caracteres, lo que es posible porque `rmsprop`, `binary_crossentropy` y 
`accuracy` están empaquetados como parte de Keras. Algunas veces tal vez quieras configurar los parámetros de tu optimizador, o pasar una función de pérdida personalizada o métricas propias. Lo primero se puede hacer pasando una instancia de una clase específica de optimizador como el argumento `optimizer`:

In [0]:
# OPCIONAL: Customizar el optimizador pasando un objeto optimizer en lugar
# de una string. Ver: https://keras.io/optimizers/ 
from keras import optimizers

## TU CÓDIGO AQUÍ ##

In [0]:
# OPCIONAL: Customizar las métricas loggueadas por el modelo.
# ver: https://keras.io/metrics/
from keras import losses, metrics

## TU CÓDIGO AQUÍ ##

## Validando nuestro enfoque

Para monitorear durante training el accuracy del modelo en datos que no ha visto antes necesitamos crear un set de validación, separando 10.000 observaciones del set de training original:

In [0]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

Vamos a entrenar nuestro modelo por 20 epochs (20 iteraciones sobre todas las muestras en los tensores de `x_train` y `y_train`), en *mini-batches* de 512 
observaciones. Al mismo tiempo monitoreamos la pérdida y el accuracy en las 10.000 observaciones que separamos. Esto se hace pasando el set de validación como el argumento de `validation_data`:

In [0]:
# Ajustar el modelo usando los siguientes parametros:
# - Para entrenar Usamos partial_x_train y partial_y_train
# - Para validar pasamos (x_val, y_val) al argumento validation_data.
# - 20 epochs
# - Batch size de 512 elementos
# 
# Asegurarse de guardar la historia del entrenamiento la cual es retornada por
# el método fit. Ver que la variable utilizada se llama "history".

history = ... ## TU CÓDIGO AQUÍ ##


En CPU, esto va a tomar menos de dos segundos por *epoch* -- el traininig lleva 20 segundos. Al final de cada *epoch* hay una leve pause en la que el modelo computa su pérdida y accuracy en las 10.000 muestras del set de validación.

Nótese que la llama a `model.fit()` devuelve un objeto `History`. Este objeto tiene un miembro `history`, que es un diccionario conteniendo datos sobre todo lo que pasó durante el entrenamiento. Echémosle un vistazo:
Echémosle un vistazo al objeto `History` retornado por `.fit`:

In [0]:
history_dict = history.history
history_dict.keys()

Contiene 4 entradas: uno por métrica que fue monitoreada, durante train y durante validation. Usemos Matplotlib para graficar la pérdida y el accuracy en train y validation.

In [0]:
import matplotlib.pyplot as plt

acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" es por "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b es por "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [0]:
plt.clf()   # figura limpia
acc_values = history_dict['binary_accuracy']
val_acc_values = history_dict['val_binary_accuracy']

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

Los puntos son la pérdida y accuracy de training, mientras que las líneas sólidas representan los resultados en validation.

Como se puede ver, la pérdida en training decrece con cada *epoch* y el accuracy de training crece con cada *epoch*. Es lo esperable al emplear optimización por el descenso del gradiente -- la cantidad que se está tratando de minimizar se vuelve más baja en cada iteración. Pero este no es el caso para la pérdida y el accuracy en validation: parecen alcanzar un pico en la cuarta epoch. 
Esto es un ejemplo de algo que advertíamos previamente: un modelo que performa mejor en el set de training no necesariamente es un modelo que va a dar mejor en datos que nunca antes ha visto. 
En términos precisos, lo que estás viendo es el llamado *overfitting*: después de la segunda *epoch* estamos sobre optimizando en el set de training, y terminamos aprendiendo representaciones que son específicas del set de training y no generalizan fuera de este set.

En este caso, para prevenir el *overfitting* podríamos simplemente frenar el training después de tres *epochs*. En general, para hacer esto hay un rango de técnicas que vamos a ir aprendiendo.

Entrenemos una nueva red desde el comienzo por cuatro *epochs*, luego evaluémosla en el set de test.

In [0]:
# Entrenar el modelo sobre todo x_train e y_train.
# Esta vez con 4 epochs.

## TU CÓDIGO AQUÍ ##


In [0]:
# Evaluamos la performance del modelo en el conjunto de test

## TU CÓDIGO AQUÍ ##

Con nuestro enfoque ingenuo alcanzamos un accuracy del 88%. Con técnicas del estado del arte se debe poder alcanzar cerca de 95%.

## Usando una red entrenada para generar predicciones sobre datos nuevos

Después de entrenar una red, vas a querer usarla. Esto se puede hacer usando el método `predict`:

In [0]:
# Usar el método predict del modelo para obtener las probabilidades estimadas
# en x_test.

## TU CÓDIGO AQUÍ ##

Como se puede ver, la red está muy "confiada" para ciertas observaciones (0.99 o más, o 0.01 o menos) pero no lo es tanto para otra (0.6, 0.4).

Ahora vamos a ver como funciona con un texto propio:


In [0]:
def text_to_vect(texts, word_to_id):
  vects = [word_to_id[w]+3 for w in texts if w in word_to_id]
  return vectorize_sequences([vects]) 

vect = text_to_vect("<START> " + "this was an excellent movie, i had a ball watching it".lower(), word_to_id)
model.predict(vect)

## Experimentos adicionales

* Estamos usando dos capas ocultas. Intentá usar 1 o 3 capas ocultas y ver cómo afecta el accuracy en validation y test.
* Intentá usar capas con más y con menos unidades oculta: 32 unidades, 64 unidades...
* Intentá usar la función de pérdida `mse` en vez de `binary_crossentropy`.
* Intentá usar la función de activación `tanh` (una función de activación que era popular en los primeros días de las redes neuronales) en vez de `relu`.

Estos experimentos te van a convencer de que la decisión de arquitectura original que hicimos es bastante razonable, aunque puede ser mejorada.

## Conclusiones

* Generalmente hay bastante de preprocesamiento que necesitás hacer en la información cruda para poder alimentar la red neuronal. En el caso de secuencias de palabras pueden ser codificadas como vectores binarios, pero hay otras opciones de codificación.
* Pilas de capas `Dense` con funciones de activación `relu` pueden resolver una gran variedad de problemas (incluyendo clasificación de sentimientos) y probablemente los uses frecuentemente.
* En un problema de clasificación binario (dos clases), tu red debe terminar en una capa `Dense` con una unidad y una función de activación `sigmoid`, 
i.e. el *output* de tu red debe ser un escalar entre 0 y 1, codificando una probabilidad.
* Con un *output* escalar que proviene de una sigmoidal, en un problema de clasificación binario, la función de pérdida que tenés que usar es `binary_crossentropy`.
* El optimizador `rmsprop` es generalmente una opción suficientemente buena, cualquier sea el problema. Es una cosa menos de qué preocuparte.
* En la medida en que se vuelven mejores en el set de training la redes neuronales eventualmente comienzan a hacer _overfitting_ y terminan obteniendo resultados crecientemente malos en data que no ha sido vista previamente. Asegurate de monitorear la *performance* en datos fuera del set de training.